In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
#Importing data
df= pd.read_csv('Website Vistiors Daywise.csv')
#Printing head
df.head()



,Date,Daily Visitors
0,01-10-2021,2649
1,02-10-2021,2000
2,03-10-2021,2011
3,04-10-2021,3219
4,05-10-2021,3870


In [2]:
#Printing tail
df.tail()

,Date,Daily Visitors
168,18-03-2022,2494
169,19-03-2022,2976
170,20-03-2022,2534
171,21-03-2022,3641
172,22-03-2022,4031


In [3]:
#Subsetting the dataset
#Index 11856 marks the end of year 2013
df = pd.read_csv('Website Vistiors Daywise.csv', nrows = 11856)

#Creating train and test set 
#Index 10392 marks the end of October 2013 
from sklearn.model_selection import train_test_split
# split the data into train and test set
train, test = train_test_split(df, test_size=0.2, random_state=3, shuffle=True)


In [4]:
train.shape

(138, 2)

In [5]:
test.shape

(35, 2)

# Naive Approach

In [6]:
dd= np.asarray(train['Daily Visitors'])
y_hat = test.copy()
y_hat['naive'] = dd[len(dd)-1]

In [7]:
from sklearn.metrics import mean_squared_error
from math import sqrt
Naive_approach_rmse = sqrt(mean_squared_error(test['Daily Visitors'], y_hat.naive))
print(Naive_approach_rmse)

899.8249671066986


# Simple Average

In [8]:
y_hat_avg = test.copy()
y_hat_avg['avg_forecast'] = train['Daily Visitors'].mean()

In [9]:
Simple_average_rmse = sqrt(mean_squared_error(test['Daily Visitors'], y_hat_avg.avg_forecast))
print(Simple_average_rmse)

802.1979163528973


# Moving Average

In [10]:
y_hat_avg = test.copy()
y_hat_avg['moving_avg_forecast'] = train['Daily Visitors'].rolling(60).mean().iloc[-1]

In [11]:
Moving_average_rmse = sqrt(mean_squared_error(test['Daily Visitors'], y_hat_avg.moving_avg_forecast))
print(Moving_average_rmse)

794.9470166032131


# #Simple Exponential Smoothing

In [12]:
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

In [13]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
y_hat_avg = test.copy()
fit2 = SimpleExpSmoothing(np.asarray(train['Daily Visitors'])).fit(smoothing_level=0.6,optimized=False)
y_hat_avg['SES'] = fit2.forecast(len(test))


C:\Users\Dell\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning: After 0.13 initialization must be handled at model creation
  warnings.warn(


In [14]:
Simple_exponential_smoothing_rmse = sqrt(mean_squared_error(test['Daily Visitors'], y_hat_avg.SES))
print(Simple_exponential_smoothing_rmse)


796.0619072154274


# #Holt’s Linear Trend method

In [15]:
y_hat_avg = test.copy()

fit1 = Holt(np.asarray(train['Daily Visitors'])).fit(smoothing_level = 0.3,smoothing_slope = 0.1)
y_hat_avg['Holt_linear'] = fit1.forecast(len(test))

C:\Users\Dell\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning: After 0.13 initialization must be handled at model creation
  warnings.warn(
<ipython-input-15-48ee11a63e62>:3: FutureWarning: the 'smoothing_slope'' keyword is deprecated, use 'smoothing_trend' instead
  fit1 = Holt(np.asarray(train['Daily Visitors'])).fit(smoothing_level = 0.3,smoothing_slope = 0.1)


In [16]:
Holt_linear_trend_method_rmse = sqrt(mean_squared_error(test['Daily Visitors'], y_hat_avg.Holt_linear))
print(Holt_linear_trend_method_rmse)

827.2488355896172


# Holt-Winters Method

In [17]:
y_hat_avg = test.copy()
fit1 = ExponentialSmoothing(np.asarray(train['Daily Visitors']) ,seasonal_periods=7 ,trend='add', seasonal='add',).fit()
y_hat_avg['Holt_Winter'] = fit1.forecast(len(test))

C:\Users\Dell\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning: After 0.13 initialization must be handled at model creation
  warnings.warn(
C:\Users\Dell\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:920: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [18]:
Holt_winters_method_rmse = sqrt(mean_squared_error(test['Daily Visitors'], y_hat_avg.Holt_Winter))
print(Holt_winters_method_rmse)

894.1429858937604


In [19]:
#Compare the results 

data = {"MODEL":pd.Series(['Naive_approach_rmse','Simple_average_rmse','Moving_average_rmse ','Simple_exponential_smoothing_rmse','Holt_linear_trend_method_rmse ','Holt_winters_method_rmse']),"RMSE_Values":pd.Series([Naive_approach_rmse,Simple_average_rmse,Moving_average_rmse ,Simple_exponential_smoothing_rmse,Holt_linear_trend_method_rmse ,Holt_winters_method_rmse])}
table_rmse=pd.DataFrame(data)
table_rmse.sort_values(['RMSE_Values'])

,MODEL,RMSE_Values
2,Moving_average_rmse,794.947017
3,Simple_exponential_smoothing_rmse,796.061907
1,Simple_average_rmse,802.197916
4,Holt_linear_trend_method_rmse,827.248836
5,Holt_winters_method_rmse,894.142986
0,Naive_approach_rmse,899.824967
